In [1]:
from medhok import get_dataset as ds

In [2]:
feats, dialects = ds.load_windowed_dataset('mel_spectrogram', split=False, onehot=True, normalised=True)

Using normalisation method: True
Loading Ngadas: 
- Marsam.wav (time: 0.9)
- Mas_Tom_Ngadas_201102.wav (time: 0.0)
Loading Banten: 
- JWBP-20121229-A.wav (time: 0.1)
- JWBA-20100703-a.wav (time: 0.1)
- JWB-Banten_9.wav (time: 0.3)
- banten-2010-07-02-a.wav (time: 0.2)
- JWB-20090125.wav (time: 0.2)
- JWBP-20121229-B.wav (time: 0.0)
- JWB-Banten_10.wav (time: 0.0)
- JWB-20090126.wav (time: 0.2)
- JWB-20090116.wav (time: 0.1)
- JWB-Banten20100605-B.wav (time: 0.0)
- JWB-Banten_1.wav (time: 0.0)
- JWB-Banten_6.wav (time: 0.1)
- JWBA-20100703-b.wav (time: 0.1)
- JWB-Banten_4.wav (time: 0.0)
- JWB-20090117.wav (time: 0.1)
- JWB-20090118.wav (time: 0.0)
- JWB-Banten_8.wav (time: 0.0)
- JWB-20090131.wav (time: 0.3)
- JWB-20090130.wav (time: 0.2)
- JWB-Banten_11.wav (time: 0.0)
- JWB-Banten_5.wav (time: 0.0)
- JWB-20090123.wav (time: 0.0)
- JWB-20090124.wav (time: 0.1)
- JWB-20090127.wav (time: 0.2)
- JWB-Banten_19.wav (time: 0.0)
- JWB-Banten20100605-A.wav (time: 0.1)
- JWB-Banten_7.wav (time

In [4]:
import pickle

total_amount = feats.shape[0]
print(f"Our dataset has {total_amount} samples.")

with open('../dataset/tfrecords/dataset_amount.pickle', 'wb') as f:
    pickle.dump(total_amount, f)

Our dataset has 41594 samples.


In [5]:
import tensorflow as tf

feats_ds = tf.data.Dataset.from_tensor_slices((feats, dialects.reshape(-1)))
feats_ds

2022-02-24 15:30:19.802021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-24 15:30:19.837828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-24 15:30:19.838091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-24 15:30:19.839089: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
for feat, dial in feats_ds.take(1):
    tf_feat = feat
    tf_dial = dial
    print(feat, dial, sep='\n')

In [ ]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_example(feats, dialects):
    feature = {
        'feature': _bytes_feature(tf.io.serialize_tensor(feats).numpy()),
        'dialect': _bytes_feature(dialects)
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def tf_serialize_example(feats, dialects):
    tf_string = tf.py_function(
        serialize_example,
        (feats, dialects),
        tf.string
    )
    return tf.reshape(tf_string, ())

In [ ]:
serialized_feature_dataset = feats_ds.map(tf_serialize_example)
serialized_feature_dataset

In [ ]:
def generator():
    for features in feats_ds:
        yield serialize_example(*features)

In [ ]:
serialized_feature_dataset = tf.data.Dataset.from_generator(
    generator,
    output_types=tf.string,
    output_shapes=()
)

In [ ]:
serialized_feature_dataset

In [ ]:
filename = '../dataset/tfrecords/mel_spectrogram_windowed_normalised.tfrecords'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_feature_dataset)

In [ ]:
tf.data.experimental.save(serialized_feature_dataset, filename)